# NJ Fair Redistricting Modified Flood Fill Algorithm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx

In [ ]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [ ]:

def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)

In [ ]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [ ]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [ ]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [ ]:
nj_current_assignment = pd.read_csv('/content/drive/MyDrive/AIS: Assignment 2 - Fair map/Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

,GEOID20,District
0,34033020001,2
1,34033001501,2
2,34033042009,2
3,34033000703,2
4,34033042001,2
...,...,...
6356,34039045302,7
6357,34039045303,7
6358,34039045404,7
6359,34039045801,7


### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [ ]:
nj_precinct_data = pd.read_csv('/content/drive/MyDrive/AIS: Assignment 2 - Fair map/Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [ ]:
shpfile = '/Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = '/Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = '//Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv.

In [ ]:
nj_contiguity = pd.read_csv('/Map_Data/Contiguity_nj.csv', header=None)

In [ ]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(12, nj_current_assignment, nj_contiguity)))

District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True


In [ ]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))


<ipython-input-6-4d60bb7804f1>:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if shape(boundaries[i]).type == 'Polygon':
<ipython-input-6-4d60bb7804f1>:7: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif shape(boundaries[i]).type == 'MultiPolygon':


D1 PP : 0.41768102211569347
D1 BR : 0.45075813446417157
D2 PP : 0.2632665176502347
D2 BR : 0.38278056561756263
D3 PP : 0.2280937682959879
D3 BR : 0.38134920642809134
D4 PP : 0.24812480573284196
D4 BR : 0.5390173747196018
D5 PP : 0.2410116694999733
D5 BR : 0.36320426268176653
D6 PP : 0.14677124653853732
D6 BR : 0.32853496486220907
D7 PP : 0.20246375771704353
D7 BR : 0.44049249082841035
D8 PP : 0.11227347882175574
D8 BR : 0.36670629634952656
D9 PP : 0.1683197710884751
D9 BR : 0.29705227593212374
D10 PP : 0.12061263370064262
D10 BR : 0.34528827672774703
D11 PP : 0.22236600778446886
D11 BR : 0.5557086439792166
D12 PP : 0.1620092442171186
D12 BR : 0.38520439164401626


In [ ]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

{1: 775340, 2: 778354, 3: 778489, 4: 767834, 5: 774454, 6: 778516, 7: 785173, 8: 800074, 9: 766863, 10: 746178, 11: 769523, 12: 768196}


# Gerneating a better and more fair redistricting map


In [ ]:

def getIsolates(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return list(nx.isolates(district_graph))

In [ ]:
# Read assignments and precint data files from csv

# Current NJ map and precint data
nj_current_assignment = pd.read_csv('/content/drive/MyDrive/AIS: Assignment 2 - Fair map/Map_Data/precinct-assignments-congress-nj.csv')
nj_precinct_data = pd.read_csv('/content/drive/MyDrive/AIS: Assignment 2 - Fair map/Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]

# Seed map and precint data
nj_seed_assignment = pd.read_csv('/content/drive/MyDrive/AIS: Assignment 2 - Fair map/Map_Data/nj_seed_assignment.csv')
nj_seed_precinct_data = pd.read_csv('/content/drive/MyDrive/AIS: Assignment 2 - Fair map/Map_Data/nj_seed_precinct_data.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_seed_precinct_data = nj_seed_precinct_data[keepcolumns]


In [ ]:
# Use seed map and create nj_current_assignment and nj_precinct_data
nj_seed_assignment = nj_seed_assignment.loc[nj_seed_assignment['District'].isin([9, 5, 10, 8, 7])]

# Set all unassigned precints' districts to -1
nj_current_assignment['District'] = -1

for index, row in nj_seed_assignment.iterrows():
    if row['GEOID20'] in nj_current_assignment['GEOID20'].values:
        nj_current_assignment.loc[nj_current_assignment['GEOID20'] == row['GEOID20'], 'District'] = row['District']
for index, row in nj_current_assignment.iterrows():
    # if row['GEOID20'] in nj_precinct_data['GEOID20'].values:
    nj_precinct_data.loc[nj_precinct_data['GEOID20'] == row['GEOID20'], 'District'] = row['District']


In [ ]:
import pandas as pd
import ast

# Seed precincts(starting precincts) for each district
initial_geoid = {
   11: "34037070001",
   9: "34031055007",
   5: "34003010001",
   8: "34017030F01",
   7: "34039045701",
   10: "34013045S02",
   6: "34023077103",
   4: "34025050001",
   3: "34005035001",
   2: "34001025001",
   1: "34015105002",
   12: "34035050010"
}

# Initialize
district_assignments = nj_current_assignment.copy()
district_precinct_data = nj_precinct_data.copy()


for district in initial_geoid:
  precint = initial_geoid[district]

precincts_added = 0
district_sizes = {i: 0 for i in range(12)}
next_precincts = [[] for _ in range(12)] # 12 lists for 12 districts
visited = set()

# Add pre painted precincts into district_sizes and visited
district_sizes[8] = len(nj_current_assignment[nj_current_assignment['District'] == 9])
district_sizes[4]= len(nj_current_assignment[nj_current_assignment['District'] == 5])
district_sizes[9] = len(nj_current_assignment[nj_current_assignment['District'] == 10])
district_sizes[7] = len(nj_current_assignment[nj_current_assignment['District'] == 8])
district_sizes[6] = len(nj_current_assignment[nj_current_assignment['District'] == 7])

for geoid in district_assignments[district_assignments['District'].isin([9, 5, 10, 8, 7])]['GEOID20']:
  visited.add(geoid)

# Set initial precincts for each district
for i in range(12):
  if i in [8, 4, 9, 7, 6]:
    continue

  # Initalize seed precint as starting precinct for each district
  initial_precinct = initial_geoid[i+1]

  # Add seed precint to the next_precincts queue for it's district
  next_precincts[i].append(initial_precinct)

  # Update district_assignments, district_sizes and visted
  district_sizes[i] = 1
  district_assignments.loc[district_assignments['GEOID20'] == initial_precinct, 'District'] = i + 1
  visited.add(initial_precinct)

# Track population of each district
district_pop_temp = getDistrictPopulations(district_assignments, district_precinct_data, 12)
district_pops = {i: int(district_pop_temp[i+1]) for i in range(12)}

# Main loop : Loops until all precincts are added to a district or all next_precincts queues are empty

while sum(district_sizes) < 6361 and any(precinct_list for precinct_list in next_precincts):

  # Find district with smallest population to add precicnts to. This ensures all districts grow at a balanced rate
  smallest_population_district = min(district_pops, key=district_pops.get)
  i = smallest_population_district
  if i in [8, 4, 9, 7, 6]:
    continue
  # Get next precinst on queue
  if next_precincts[i]:
      current_precinct = next_precincts[i].pop(0)
      precincts_added += 1

      # Print algorithm progress every 100 added precincts and save district_assignments as csv in case of cashing
      if precincts_added % 100 == 0:
        print(f"Added {precincts_added} precincts")
        print(getDistrictPopulations(district_assignments, district_precinct_data, 12))
        district_assignments.to_csv('ff_10_nj_assignment.csv', index=False)

      # For run time reasons we stop after 3400 added precincts and fill in unassigned precincts in next part
      if precincts_added == 3400:
        district_assignments.to_csv('ff_10_nj_assignment.csv', index=False)
        print(f"3400 precincts_added: break ")
        break

      # Check for contiguity and population balance
      # If not contiguous revert the precinct assignement and continue loop
      if not (district_precinct_data[district_precinct_data['GEOID20'].isin(district_assignments[district_assignments['District']==i+1]['GEOID20'])]['Total_2020_Total'].sum() < 773590 and isDistrictContiguous(i + 1, district_assignments, nj_contiguity)):
          print(f"is_balanced: {district_precinct_data[district_precinct_data['GEOID20'].isin(district_assignments[district_assignments['District']==i+1]['GEOID20'])]['Total_2020_Total'].sum() < 773590}")
          print(f"is_contiguous: {isDistrictContiguous(i + 1, district_assignments, nj_contiguity)}")
          print(f"Removing precinct {precincts_added} from district {i + 1}: {current_precinct}")
          district_assignments.loc[district_assignments['GEOID20'] == current_precinct, 'District'] = -1
          visited.remove(current_precinct)
          precincts_added -= 1
          district_pops[i] -= int(district_precinct_data[district_precinct_data['GEOID20'] == current_precinct]['Total_2020_Total'])
          continue

      # If contiguous and and population balanced, find and add neighbors to district's next_precincts queue
      neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct'] == current_precinct]['Neighbors'].values[0])
      for neighbor in neighbors:
          if neighbor not in visited and neighbor not in next_precincts[i]:
              next_precincts[i].append(neighbor)

              # Update district_assignments, visited, district_sizes, and district_pops
              district_assignments.loc[district_assignments['GEOID20'] == neighbor, 'District'] = i + 1
              visited.add(neighbor)
              district_sizes[i] += 1
              district_pops[i] += int(district_precinct_data[district_precinct_data['GEOID20'] == neighbor]['Total_2020_Total'])


# Export final assignments: This export will have unassigned precincts
# district_assignments.to_csv('fair_redistricting.csv', index=False)



Added 100 precincts
{1: 55125, 2: 53997, 3: 54280, 4: 64451, 5: 773114, 6: 54152, 7: 772475, 8: 771203, 9: 771521, 10: 771227, 11: 57556, 12: 53975}
Added 200 precincts
{1: 92311, 2: 91946, 3: 91695, 4: 98800, 5: 773114, 6: 89749, 7: 772475, 8: 771203, 9: 771521, 10: 771227, 11: 88816, 12: 90303}
Added 300 precincts
{1: 126370, 2: 123284, 3: 118382, 4: 118296, 5: 773114, 6: 118660, 7: 772475, 8: 771203, 9: 771521, 10: 771227, 11: 118946, 12: 119737}
Added 400 precincts
{1: 148181, 2: 152867, 3: 150500, 4: 147139, 5: 773114, 6: 150394, 7: 772475, 8: 771203, 9: 771521, 10: 771227, 11: 146570, 12: 148704}
Added 500 precincts
{1: 170949, 2: 171542, 3: 176004, 4: 180867, 5: 773114, 6: 175141, 7: 772475, 8: 771203, 9: 771521, 10: 771227, 11: 170516, 12: 170941}
Added 600 precincts
{1: 196692, 2: 232549, 3: 196907, 4: 196600, 5: 773114, 6: 201220, 7: 772475, 8: 771203, 9: 771521, 10: 771227, 11: 200297, 12: 202129}
Added 700 precincts
{1: 225678, 2: 232549, 3: 226156, 4: 227098, 5: 773114, 6:

In [ ]:
# Fill in unassigned precincts
#district_assignments = pd.read_csv('/content/raw_team_5_fair_map_assignment.csv')

district_assignments['District']

# Find all unassigned_precincts
unassigned_precincts = district_assignments.loc[district_assignments['District'] == -1]

# Find all unassigned_precincts find bordering districts and assign it to the district with smallest population
border_precincts = []

# While there are unassigned precincts
while len(unassigned_precincts) != 0:
  for _, row in unassigned_precincts.iterrows():
      precinct_id = row['GEOID20']
      neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct'] == precinct_id]['Neighbors'].values[0])

      # Collect neighboring districts
      neighbor_districts = [district_assignments[district_assignments['GEOID20'] == n]['District'].values[0] for n in neighbors if district_assignments[district_assignments['GEOID20'] == n]['District'].values[0] != -1]

      # If there are neighboring districts, pick the most common one
      if neighbor_districts:

          # Assign to the nieghor district to smallest population
          neighbor_district_populations = {district: district_precinct_data[district_precinct_data['GEOID20'].isin(district_assignments[district_assignments['District']==district]['GEOID20'])]['Total_2020_Total'].sum() for district in neighbor_districts}
          smallest_population_neighbor_district = min(neighbor_district_populations, key=neighbor_district_populations.get)

          # most_common_neighbor_district = max(set(neighbor_districts), key=neighbor_districts.count)
          border_precincts.append((precinct_id, smallest_population_neighbor_district))

  unassigned_precincts = district_assignments.loc[district_assignments['District'] == -1]
  print(f"Number of unassigned precincts : {len(unassigned_precincts)}")

  for precinct_id, neighbor_district in border_precincts:
      district_assignments.loc[district_assignments['GEOID20'] == precinct_id, 'District'] = neighbor_district
      district_assignments.loc[district_assignments['GEOID20'] == precinct_id, 'District'] = neighbor_district

  # Check if all districts are contiguous
  isolates = []
  for k in range(12):
    # if not contiguous find all isolates
    if not (isDistrictContiguous(k+1, district_assignments, nj_contiguity)):
      print(f"D{k+1} is not contiguous")
      isolates = getIsolates(k+1, district_assignments, nj_contiguity)

  # Set all isolates as unassigned
  for isolate in isolates:
    district_assignments.loc[district_assignments['GEOID20'] == isolate, 'District'] = -1

district_assignments.to_csv('team_5_fair_map_assignment.csv', index=False)

Number of unassigned precincts : 206
Number of unassigned precincts : 51
D10 is not contiguous
Number of unassigned precincts : 14
Number of unassigned precincts : 1
Number of unassigned precincts : 0


Generating a redistrictign fair map

Shreeti Patel